In [13]:
import numpy as np
import torch
from transformers import AutoTokenizer, CamembertForSequenceClassification, Trainer

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

LABEL_NAMES = ['scolomfr-voc-022-num-004', 'scolomfr-voc-022-num-005', 'scolomfr-voc-022-num-006', 'scolomfr-voc-022-num-007', 'scolomfr-voc-022-num-010', 'scolomfr-voc-022-num-011', 'scolomfr-voc-022-num-013', 'scolomfr-voc-022-num-014', 'scolomfr-voc-022-num-015', 'scolomfr-voc-022-num-018', 'scolomfr-voc-022-num-020', 'scolomfr-voc-022-num-021', 'scolomfr-voc-022-num-023', 'scolomfr-voc-022-num-608', 'scolomfr-voc-022-num-129', 'scolomfr-voc-022-num-131', 'scolomfr-voc-022-num-132', 'scolomfr-voc-022-num-133', 'scolomfr-voc-022-num-134', 'scolomfr-voc-022-num-135', 'scolomfr-voc-022-num-136', 'scolomfr-voc-022-num-138', 'scolomfr-voc-022-num-201', 'scolomfr-voc-022-num-043', 'scolomfr-voc-022-num-044', 'scolomfr-voc-022-num-047', 'scolomfr-voc-022-num-048', 'scolomfr-voc-022-num-049', 'scolomfr-voc-022-num-083', 'scolomfr-voc-022-num-212', 'scolomfr-voc-022-num-027', 'scolomfr-voc-022-num-089', 'scolomfr-voc-022-num-090', 'scolomfr-voc-022-num-213', 'scolomfr-voc-022-num-095', 'scolomfr-voc-022-num-096', 'scolomfr-voc-022-num-097', 'scolomfr-voc-022-num-098', 'scolomfr-voc-022-num-153', 'scolomfr-voc-022-num-154', 'scolomfr-voc-022-num-099', 'scolomfr-voc-022-num-100', 'scolomfr-voc-022-num-103', 'scolomfr-voc-022-num-104', 'scolomfr-voc-022-num-288', 'scolomfr-voc-022-num-298', 'scolomfr-voc-022-num-231', 'scolomfr-voc-022-num-125', 'scolomfr-voc-022-num-126', 'scolomfr-voc-022-num-238', 'scolomfr-voc-022-num-640', 'scolomfr-voc-022-num-641', 'scolomfr-voc-022-num-650', 'scolomfr-voc-022-num-139', 'scolomfr-voc-022-num-146', 'scolomfr-voc-022-num-150', 'scolomfr-voc-022-num-151', 'scolomfr-voc-022-num-185', 'scolomfr-voc-022-num-187', 'scolomfr-voc-022-num-101', 'scolomfr-voc-022-num-102', 'scolomfr-voc-022-num-111', 'scolomfr-voc-022-num-112', 'scolomfr-voc-022-num-109', 'scolomfr-voc-022-num-110', 'scolomfr-voc-022-num-163', 'scolomfr-voc-022-num-164', 'scolomfr-voc-022-num-063']

FRENCH_LABELS = {"http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-002": "école maternelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-008": "école élémentaire",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-016": "collège",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-065": "voie professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-066": "voie générale et technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-140": "adaptation scolaire et scolarisation des élèves handicapés",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-141": "collège professionnel",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-142": "lycée général et technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-163": "formation des personnels de l'éducation nationale",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-164": "formation professionnelle continue des adultes",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-165": "lycée professionnel",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-185": "licence professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-195": "section BTS",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-198": "section DUT",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-226": "voie CPGE",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-239": "11-15 ans",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-240": "15-18 ans",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-241": "3-6 ans",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-242": "6-11 ans",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-615": "cycles de l'enseignement scolaire (2016)",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-063": "formation continue des personnels de l'éducation nationale",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-136": "2de générale et technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-146": "1re générale",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-201": "terminale générale",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-211": "terminale technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-134": "1re ST2S",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-123": "1re générale et technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-083": "terminale STMG",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-047": "terminale STI2D",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-043": "terminale ST2S",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-044": "terminale STD2A",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-048": "terminale STL",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-641": "terminale STHR",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-650": "2de STHR",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-132": "1re STL",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-237": "1re technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-620": "cycle 3 (2016)",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-621": "cycle 4 (2016)",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-619": "cycle 2 (2016)",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-027": "2de professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-050": "terminale professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-127": "1re professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-223": "voie CAP",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-133": "1re STMG",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-129": "1re STD2A",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-131": "1re STI2D",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-088": "terminale générale et technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-617": "cycle 1 (2016)",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-640": "1re STHR",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-162": "enseignement supérieur en lycée",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-214": "voie BMA",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-221": "voie BT",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-222": "voie BTM",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-187": "diplôme de comptabilité et gestion",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-212": "terminale TMD",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-238": "1re TMD",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-023": "3e",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-618": "cycles de la scolarité obligatoire",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-004": "toute petite section",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-005": "petite section",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-006": "moyenne section",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-007": "grande section",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-003": "cycle 1",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-009": "cycle 2",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-010": "CP",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-011": "CE1",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-013": "CE2",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-014": "CM1",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-015": "CM2",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-012": "cycle 3",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-018": "6e",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-020": "5e",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-021": "4e",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-017": "cycle d'adaptation",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-019": "cycle central",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-022": "cycle d'orientation",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-608": "3e d'insertion",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-084": "classe de détermination",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-040": "terminale ES",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-041": "terminale L",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-042": "terminale S",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-085": "cycle terminal",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-233": "voie technologique",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-049": "terminale techno hôtellerie",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-089": "terminale pro domaine de la production",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-090": "terminale pro domaine des services",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-095": "BMA - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-096": "BMA - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-097": "BP - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-098": "BP - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-099": "BTM - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-100": "BTM - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-103": "CAP - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-104": "CAP - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-125": "1re pro domaine de la production",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-126": "1re pro domaine des services",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-138": "3e professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-139": "4e professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-213": "voie BEP",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-215": "voie BP",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-227": "voie FC",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-231": "voie MC",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-288": "FC post niveau IV",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-298": "FC post niveau V",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-230": "voie générale",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-135": "1re techno hôtellerie",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-094": "cycles de l'enseignement scolaire",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-101": "BTS - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-102": "BTS - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-109": "CPGE - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-110": "CPGE - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-111": "DUT - 1re année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-112": "DUT - 2e année",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-121": "tranches d'âge de l'enseignement scolaire",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-626": "Niveau éducatif détaillé (2015-)",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-122": "1re ES",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-124": "1re L",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-128": "1re S",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-150": "ASSEH - 1er degré",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-151": "ASSEH - 2d degré",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-153": "BT - première",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-154": "BT - terminale",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-234": "voies générale, technologique, professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-284": "formation professionnelle",
                 "http://data.education.fr/voc/scolomfr/concept/scolomfr-voc-022-num-616": "cycle unique pour l'école maternelle"}

MODEL_PATH = '/home/joachim/gdrive/domain-helper/level-results/checkpoint-89796/'

tokenizer = tokenizer = AutoTokenizer.from_pretrained("camembert-base")

sample = 'Apprendre à lire avec le lapin'

X_test_tokenized = tokenizer([sample], padding="max_length", truncation=True, max_length=128)

dataset = Dataset(X_test_tokenized)
# Load trained model
model = CamembertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=len(LABEL_NAMES))

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, a, b = test_trainer.predict(dataset)

# Preprocess raw predictions
Y_pred = np.argmax(raw_pred, axis=1)[0]
positive_elements = np.where(raw_pred>0)[1]
print(np.argmax(raw_pred, axis=1))
print(LABEL_NAMES[Y_pred])
for Y in positive_elements:
    print(FRENCH_LABELS[f'http://data.education.fr/voc/scolomfr/concept/{LABEL_NAMES[Y]}'])

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/camembert-base/resolve/main/config.json from cache at /home/joachim/.cache/huggingface/transformers/f459e43c5ebb871abbf9209195563bff6a11547fd9532047739667c394833221.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.1

[9]
scolomfr-voc-022-num-018
CM1
CM2
6e
5e
